# Import Library

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import math
import os
import time
import sys
import re
from string import punctuation
from nltk import word_tokenize
from collections import Counter
from numpy import linalg as LA
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Import Database

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/data Soap Indo.xlsx')
data

,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",NaN,<p>paracetamol</p> <p>ranitidin</p> <p>imbos...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>febris hari 2</p>
1,<p>kontrol post opname</p>,NaN,<p>R/ Lycalvit syr 1x 1 cth&nbsp;</p>,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>ISK</p> <p>Post rawat inap&nbsp;</p>
2,<p>kontrol post opname</p>,NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>ISK</p> <p>Post rawat inap&nbsp;</p>
3,<p>imunisasi</p>,NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",NaN
4,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Obs febris hr ke 3 ec&nbsp;</p>
...,...,...,...,...,...
126478,<p>IMUNISASI HEPATITIS</p> <p>Anti HBs Ag (-)...,NaN,<p>Hepavax B/ Engerix B</p>,"<p>Suhu : C,Nadi : 97 x/menit,Respirasi : x/me...",<p>Vaksinasi HEp B</p>
126479,"<p>BAWAH DADA KIRI SAKIT,KADANG KADANG NYERI P...",NaN,<p>Lancid mg 30 no X<br />S2ddI</p> <p>Aldisa...,"<p>Suhu : C,Nadi : 79 x/menit,Respirasi : x/me...",<p>Dyspepsia</p>
126480,"<p>-telinga kiri ndenging sekitar 3 hari, kura...",NaN,<p>Carbogliserin s.3.d.d.gtt IV AS Lg I</p>,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>AS cerumen prop</p> <p>&nbsp;</p>
126481,"<p>TELINGA KANAN KELUAR CAIRAN, 4 hari. batpil...",<p>evakuasi discharge AD</p>,<p>Claneksi syr s.3.d.d. Ctakar I Lg I</p> <p...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>AD OMA perforasi</p>


In [ ]:
data_index = data[['FS_ANAMNESA', 'FS_DIAGNOSA']]

In [ ]:
typo = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/Check Typo.xlsx')
typo

,kata_typo,kata_benar
0,peerut,perut
1,jln,jalan
2,kakan,kanan
3,umroih,umroh
4,kleuhan,keluhan
...,...,...
1285,shdh,sudah
1286,muntahdemampusing,muntah demam pusing
1287,gejla,gejala
1288,ebih,lebih


# Preprocessing

## Missing Value

In [ ]:
# check NaN
percent_missing = data_index.isnull().sum() * 100 / len(data)
percent_missing

FS_ANAMNESA    3.335626
FS_DIAGNOSA    1.093428
dtype: float64

In [ ]:
# displays NaN in the columns FS_ANAMNESA & FS_DIAGNOSA

index = data_index[data_index.isna().any(axis=1)]
index

,FS_ANAMNESA,FS_DIAGNOSA
3,<p>imunisasi</p>,NaN
82,NaN,<p>36 KARIES</p>
84,NaN,<p>36 KARIES</p>
110,NaN,NaN
180,NaN,<p>48 PULPITIS</p>
...,...,...
126308,NaN,<p>ISPA</p> <p>Dyspepsia</p>
126309,NaN,<p>Dyspepsia</p>
126323,NaN,<p>Gaswtropathy drug induced</p>
126360,<p>KONTROL RUTIN DM</p> <p>GDS 302</p>,NaN


### Drop Missing Value

In [ ]:
data = data.dropna(subset = ['FS_ANAMNESA', 'FS_DIAGNOSA'])
data = data.reset_index(drop=True)

In [ ]:
data_index = data[['FS_ANAMNESA', 'FS_DIAGNOSA']]

In [ ]:
data_id = data_index.index

In [ ]:
data_id

RangeIndex(start=0, stop=120986, step=1)

In [ ]:
data_index = data_index.iloc[:100]
data_index

,FS_ANAMNESA,FS_DIAGNOSA
0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",<p>febris hari 2</p>
1,<p>kontrol post opname</p>,<p>ISK</p> <p>Post rawat inap&nbsp;</p>
2,<p>kontrol post opname</p>,<p>ISK</p> <p>Post rawat inap&nbsp;</p>
3,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",<p>Obs febris hr ke 3 ec&nbsp;</p>
4,<p>kontrol</p>,<p>kontrol BKB alergi&nbsp;</p>
...,...,...
95,<p>-kontrol</p>,<p>striktur uretra</p>
96,<p>-kontrol post op</p> <p>tak ada keluhan</p>,<p>stricture uretra</p>
97,<p>KONTROL sempat membaik&nbsp;</p>,<p>Dermatitis Numular&nbsp;</p>
98,<p>GATAL DI PAHA makin banyak&nbsp; 1th&nbsp;<...,<p>Dermatitis Numular&nbsp;</p>


## Lowercase

In [ ]:
# Converting the words in the dataset into lowercase format

data_index_anam = data_index['FS_ANAMNESA'].apply(lambda x: x.lower())
data_index_diag = data_index['FS_DIAGNOSA'].apply(lambda x: x.lower())

In [ ]:
# Displays for the anamnesa column

data_index_anam

0     <p>demam 2 hari, pusing, ma/mi baik</p>  <p>le...
1                            <p>kontrol post opname</p>
2                            <p>kontrol post opname</p>
3      <p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>
4                                        <p>kontrol</p>
                            ...                        
95                                      <p>-kontrol</p>
96      <p>-kontrol post op</p>  <p>tak ada keluhan</p>
97                  <p>kontrol sempat membaik&nbsp;</p>
98    <p>gatal di paha makin banyak&nbsp; 1th&nbsp;<...
99    <p>-kulit merah, gatal di kaki dan tangan&nbsp...
Name: FS_ANAMNESA, Length: 100, dtype: object

In [ ]:
# Displays 100 rows
df_lower = pd.concat([data_index_anam, data_index_diag], axis=1)
df_lower

,FS_ANAMNESA,FS_DIAGNOSA
0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",<p>febris hari 2</p>
1,<p>kontrol post opname</p>,<p>isk</p> <p>post rawat inap&nbsp;</p>
2,<p>kontrol post opname</p>,<p>isk</p> <p>post rawat inap&nbsp;</p>
3,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",<p>obs febris hr ke 3 ec&nbsp;</p>
4,<p>kontrol</p>,<p>kontrol bkb alergi&nbsp;</p>
...,...,...
95,<p>-kontrol</p>,<p>striktur uretra</p>
96,<p>-kontrol post op</p> <p>tak ada keluhan</p>,<p>stricture uretra</p>
97,<p>kontrol sempat membaik&nbsp;</p>,<p>dermatitis numular&nbsp;</p>
98,<p>gatal di paha makin banyak&nbsp; 1th&nbsp;<...,<p>dermatitis numular&nbsp;</p>


## Remove Tag HTML

In [ ]:
# remove tag html from 2 columns 

df_tag_removed = df_lower.replace(r'<[^<>]*>', '', regex=True)
df_tag_removed

,FS_ANAMNESA,FS_DIAGNOSA
0,"demam 2 hari, pusing, ma/mi baik lemas(-)",febris hari 2
1,kontrol post opname,isk post rawat inap&nbsp;
2,kontrol post opname,isk post rawat inap&nbsp;
3,"panas ,&nbsp;3 hari, miuntah, pilek&nbsp;",obs febris hr ke 3 ec&nbsp;
4,kontrol,kontrol bkb alergi&nbsp;
...,...,...
95,-kontrol,striktur uretra
96,-kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik&nbsp;,dermatitis numular&nbsp;
98,gatal di paha makin banyak&nbsp; 1th&nbsp; &n...,dermatitis numular&nbsp;


## Remove Punctuation

In [ ]:
# remove punct except between number from 2 columns

df_punct_exc_number = df_tag_removed.replace(r'(?<!\d)[.,;:+](?!\d)', '', regex=True)
df_punct_exc_number

,FS_ANAMNESA,FS_DIAGNOSA
0,demam 2 hari pusing ma/mi baik lemas(-),febris hari 2
1,kontrol post opname,isk post rawat inap&nbsp
2,kontrol post opname,isk post rawat inap&nbsp
3,panas &nbsp;3 hari miuntah pilek&nbsp,obs febris hr ke 3 ec&nbsp
4,kontrol,kontrol bkb alergi&nbsp
...,...,...
95,-kontrol,striktur uretra
96,-kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik&nbsp,dermatitis numular&nbsp
98,gatal di paha makin banyak&nbsp 1th&nbsp &nbs...,dermatitis numular&nbsp


In [ ]:
df_punct_exc_number = df_punct_exc_number.replace(r'[(_)!?@/=;:#$*^~{}\-]', ' ', regex=True)
df_punct_exc_number

,FS_ANAMNESA,FS_DIAGNOSA
0,demam 2 hari pusing ma mi baik lemas,febris hari 2
1,kontrol post opname,isk post rawat inap&nbsp
2,kontrol post opname,isk post rawat inap&nbsp
3,panas &nbsp 3 hari miuntah pilek&nbsp,obs febris hr ke 3 ec&nbsp
4,kontrol,kontrol bkb alergi&nbsp
...,...,...
95,kontrol,striktur uretra
96,kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik&nbsp,dermatitis numular&nbsp
98,gatal di paha makin banyak&nbsp 1th&nbsp &nbs...,dermatitis numular&nbsp


In [ ]:
df_punct_exc_number = df_punct_exc_number.replace(r'[.,;:+-?!(_)@/#$*^~{}](?!\d\w)',' ', regex=True)
df_punct_exc_number

## Remove Stopwords

### FS_ANAMNESA

In [ ]:
# remove stopwords from column FS_ANAMNESA

df_remove_sw = df_punct_exc_number['FS_ANAMNESA']
dict_doc_anam = []

def filtered(doc):
  open_stopwords = open('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/stopwords.txt', 'r')
  stopwords = open_stopwords.read()
  stopwords = stopwords.split("\n")
  word_tokens = word_tokenize(doc)
  filtered_sentence = [w for w in word_tokens if not w in stopwords]
  return ' '.join(filtered_sentence)

def remove_stopwords(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    dict_doc_anam.append(data)
  return dict_doc_anam

remove_stopwords(df_remove_sw)

['demam 2 hari pusing ma mi baik lemas',
 'kontrol post opname',
 'kontrol post opname',
 'panas & 3 hari miuntah pilek &',
 'kontrol',
 'batuk panas pilek &',
 'batuk sering kambuh',
 'batuk pilek demam kemerahan badan',
 'batuk sesak napas &',
 'rujukan klinik laras hati gigi kiri atas berlubang',
 'melanjutkan perawatan gigi sebelah kiri atas',
 'melanjutkan perawatan',
 'pusing lemas batuk demam 1 hari dahak warna kuning kental paracetamol tidak membaik',
 'minta surat keterangan sehat 7 bebas narkoba',
 'kontrol evaluasi bmp',
 'kontrol 4',
 'kontrol et konsultasi hasil laborat',
 'kontrol',
 'kontrol et',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol te',
 'kontrol',
 'kontrol',
 'kontrol',
 'kontrol te',
 'kontrol pembesaran kelenjar getah bening leher',
 'g4p3ab1 hpm 11 06 18 kellkadang nyeri perut bagian bawah',
 'kadang kenceng kencenghpm 11 juni 18.uk23+ 5mg',
 'g4p2a1 hpht 11 6 2018 oeriksa hamil t a k',
 'g4p2a1 , hpht 11 6 19 , periksa kehamilan',
 'g4p2a1 , hpht 1

In [ ]:
# Change the column FS_ANAMNESA format and display in series form 

df_remove_sw_an = pd.Series(dict_doc_anam)
df_remove_sw_an

0           demam 2 hari pusing ma mi baik lemas
1                            kontrol post opname
2                            kontrol post opname
3                 panas & 3 hari miuntah pilek &
4                                        kontrol
                         ...                    
95                                       kontrol
96               kontrol post op tak ada keluhan
97                      kontrol sempat membaik &
98    gatal paha makin banyak & 1th & & nbsptx &
99      kulit merah gatal kaki tangan & & 2 th &
Length: 100, dtype: object

### FS_DIAGNOSA

In [ ]:
# remove stopwords from column FS_DIAGNOSA

df_remove_sw = df_punct_exc_number['FS_DIAGNOSA']
dict_doc_diag = []

def filtered(doc):
  open_stopwords = open('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/stopwords.txt', 'r')
  stopwords = open_stopwords.read()
  stopwords = stopwords.split("\n")
  word_tokens = word_tokenize(doc)
  filtered_sentence = [w for w in word_tokens if not w in stopwords]
  return ' '.join(filtered_sentence)

def remove_stopwords(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    dict_doc_diag.append(data)
  return dict_doc_diag

remove_stopwords(df_remove_sw)

['febris hari 2',
 'isk post rawat inap &',
 'isk post rawat inap &',
 'obs febris 3 ec &',
 'kontrol bkb alergi &',
 'rfa',
 'bkb ec alergi susp asma',
 'scarlet fever &',
 'asma bronkhiale serang &',
 'gigi 65 nv gingival polyp',
 'gigi 65 nv',
 '65 nv',
 'common cold',
 'sehat bebas narkoba',
 'et',
 'thrombositosis esensial limfadenopati reaktif &',
 'et &',
 'et',
 'et',
 'et &',
 'et',
 'et',
 'et &',
 'et',
 'mpd esential thrombositosis',
 'et',
 'te',
 'tromvbositosis esensial &',
 'te',
 '6 7 mg',
 'hamil 23 mg',
 'hamil 37 mg',
 'hamil 36 mg preskep',
 'hamil 27 mg vitamin diteruskan',
 'hamil 18 mg',
 'hamil 19 mg edd usg 25 8 2019',
 'konsultasi ia turunkan bb & suami pernah minum cc post op varicocele & bulan juni permata hati',
 'calculus penundaan haid',
 'calculus',
 'dermatitis atopik &',
 'dermatitis atopik',
 'sun burn & & nbspdermatitis foto kontak alergika &',
 'dki & nbsppityriasis alba &',
 'pitted keratolysis',
 'dermatitis atopik &',
 'pompholyx',
 'tinea korpo

In [ ]:
# Change the column FS_DIAGNOSA format and display in series form

df_remove_sw_diag = pd.Series(dict_doc_diag)
df_remove_sw_diag

0                                  febris hari 2
1                          isk post rawat inap &
2                          isk post rawat inap &
3                              obs febris 3 ec &
4                           kontrol bkb alergi &
                         ...                    
95                               striktur uretra
96                              stricture uretra
97                          dermatitis numular &
98                          dermatitis numular &
99    dermatitis asteatotik psoriasis vulgaris &
Length: 100, dtype: object

In [ ]:
df_remove_sw = pd.concat([df_remove_sw_an, df_remove_sw_diag], axis=1)
df_remove_sw

,0,1
0,demam 2 hari pusing ma mi baik lemas,febris hari 2
1,kontrol post opname,isk post rawat inap &
2,kontrol post opname,isk post rawat inap &
3,panas & 3 hari miuntah pilek &,obs febris 3 ec &
4,kontrol,kontrol bkb alergi &
...,...,...
95,kontrol,striktur uretra
96,kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik &,dermatitis numular &
98,gatal paha makin banyak & 1th & & nbsptx &,dermatitis numular &


In [ ]:
df_remove_sw.columns =['FS_ANAMNESA', 'FS_DIAGNOSA']

In [ ]:
df_remove_sw

,FS_ANAMNESA,FS_DIAGNOSA
0,demam 2 hari pusing ma mi baik lemas,febris hari 2
1,kontrol post opname,isk post rawat inap &
2,kontrol post opname,isk post rawat inap &
3,panas & 3 hari miuntah pilek &,obs febris 3 ec &
4,kontrol,kontrol bkb alergi &
...,...,...
95,kontrol,striktur uretra
96,kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik &,dermatitis numular &
98,gatal paha makin banyak & 1th & & nbsptx &,dermatitis numular &


## Remove Multiple Whitespace

In [ ]:
# remove multiple whitespace from FS_ANAMNESA

df_content_anam = df_remove_sw['FS_ANAMNESA'].replace(r'\s\s+', ' ')
df_content_anam

0           demam 2 hari pusing ma mi baik lemas
1                            kontrol post opname
2                            kontrol post opname
3                 panas & 3 hari miuntah pilek &
4                                        kontrol
                         ...                    
95                                       kontrol
96               kontrol post op tak ada keluhan
97                      kontrol sempat membaik &
98    gatal paha makin banyak & 1th & & nbsptx &
99      kulit merah gatal kaki tangan & & 2 th &
Name: FS_ANAMNESA, Length: 100, dtype: object

In [ ]:
# remove multiple whitespace from FS_DIAGNOSA

df_content_diag = df_remove_sw.FS_DIAGNOSA.replace(r'\s\s+', ' ')
df_content_diag

0                                  febris hari 2
1                          isk post rawat inap &
2                          isk post rawat inap &
3                              obs febris 3 ec &
4                           kontrol bkb alergi &
                         ...                    
95                               striktur uretra
96                              stricture uretra
97                          dermatitis numular &
98                          dermatitis numular &
99    dermatitis asteatotik psoriasis vulgaris &
Name: FS_DIAGNOSA, Length: 100, dtype: object

In [ ]:
# concatenate 2 columns

df_content = pd.concat([df_content_anam, df_content_diag], axis=1)
df_content

,FS_ANAMNESA,FS_DIAGNOSA
0,demam 2 hari pusing ma mi baik lemas,febris hari 2
1,kontrol post opname,isk post rawat inap &
2,kontrol post opname,isk post rawat inap &
3,panas & 3 hari miuntah pilek &,obs febris 3 ec &
4,kontrol,kontrol bkb alergi &
...,...,...
95,kontrol,striktur uretra
96,kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik &,dermatitis numular &
98,gatal paha makin banyak & 1th & & nbsptx &,dermatitis numular &


## Replacement Typo Word

### FS_ANAMNESA

In [ ]:
# Replacement typo word

content_anam = []

def filtered(doc):
  word_tokens = word_tokenize(doc)
  for x in range(len(word_tokens)):
    for y in range(len(typo)):
      if word_tokens[x] == typo['kata_typo'][y]:
        word_tokens[x] = typo['kata_benar'][y]
      else:
        continue
  return ' '.join(word_tokens)

def replacement(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    content_anam.append(data)
  return content_anam

In [ ]:
# Replacement typo in column FS_ANAMNESA
replacement(df_content['FS_ANAMNESA'])

['demam 2 hari pusing makan minum baik lemas',
 'kontrol post opname',
 'kontrol post opname',
 'panas & 3 hari miuntah pilek &',
 'kontrol',
 'batuk panas pilek &',
 'batuk sering kambuh',
 'batuk pilek demam kemerahan badan',
 'batuk sesek nafas &',
 'rujukan klinik laras hati gigi kiri atas berlubang',
 'melanjutkan perawatan gigi sebelah kiri atas',
 'melanjutkan perawatan',
 'pusing lemas batuk demam 1 hari dahak warna kuning kental paracetamol tidak membaik',
 'minta surat keterangan sehat 7 bebas narkoba',
 'kontrol evaluasi bmp',
 'kontrol 4',
 'kontrol et konsultasi hasil laborat',
 'kontrol',
 'kontrol et',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol te',
 'kontrol',
 'kontrol',
 'kontrol',
 'kontrol te',
 'kontrol pembesaran kelenjar getah bening leher',
 'g4p3ab1 hpm 11 06 18 kellkadang nyeri perut bagian bawah',
 'kadang kenceng kencenghpm 11 juni 18.uk23+ 5mg',
 'g4p2a1 hpht 11 6 2018 oeriksa hamil t a k',
 'g4p2a1 , hpht 11 6 19 , periksa kehamilan',
 'g4p2a1 , 

### FS_DIAGNOSA

In [ ]:
#replacement typo word

content_diag = []

def filtered(doc):
  word_tokens = word_tokenize(doc)
  for x in range(len(word_tokens)):
    for y in range(len(typo)):
      if word_tokens[x] == typo['kata_typo'][y]:
        word_tokens[x] = typo['kata_benar'][y]
      else:
        continue
  return ' '.join(word_tokens)

def replacement(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    content_diag.append(data)
  return content_diag

In [ ]:
replacement(df_content['FS_DIAGNOSA'])

['febris hari 2',
 'isk post rawat inap &',
 'isk post rawat inap &',
 'obs febris 3 ec &',
 'kontrol bkb alergi &',
 'rfa',
 'bkb ec alergi susp asma',
 'scarlet fever &',
 'asma bronkhiale serang &',
 'gigi 65 non vital gingival polyp',
 'gigi 65 non vital',
 '65 non vital',
 'common cold',
 'sehat bebas narkoba',
 'et',
 'thrombositosis esensial limfadenopati reaktif &',
 'et &',
 'et',
 'et',
 'et &',
 'et',
 'et',
 'et &',
 'et',
 'mpd esential thrombositosis',
 'et',
 'te',
 'tromvbositosis esensial &',
 'te',
 '6 7 mg',
 'hamil 23 mg',
 'hamil 37 mg',
 'hamil 36 mg preskep',
 'hamil 27 mg vitamin diteruskan',
 'hamil 18 mg',
 'hamil 19 mg edd usg 25 8 2019',
 'konsultasi ia turunkan bb & suami pernah minum cc post op varicocele & bulan juni permata hati',
 'calculus penundaan haid',
 'calculus',
 'dermatitis atopik &',
 'dermatitis atopik',
 'sun burn & & nbspdermatitis foto kontak alergika &',
 'dki & nbsppityriasis alba &',
 'pitted keratolysis',
 'dermatitis atopik &',
 'pomp

### Concatenantion

In [ ]:
content_anam = pd.Series(content_anam)
content_diag = pd.Series(content_diag)

In [ ]:
content_anam

0      demam 2 hari pusing makan minum baik lemas
1                             kontrol post opname
2                             kontrol post opname
3                  panas & 3 hari miuntah pilek &
4                                         kontrol
                         ...                     
95                                        kontrol
96                kontrol post op tak ada keluhan
97                       kontrol sempat membaik &
98     gatal paha makin banyak & 1th & & nbsptx &
99    kulit merah gatal kaki tangan & & 2 tahun &
Length: 100, dtype: object

In [ ]:
df_content = pd.concat([content_anam, content_diag], axis=1)
df_content

,0,1
0,demam 2 hari pusing makan minum baik lemas,febris hari 2
1,kontrol post opname,isk post rawat inap &
2,kontrol post opname,isk post rawat inap &
3,panas & 3 hari miuntah pilek &,obs febris 3 ec &
4,kontrol,kontrol bkb alergi &
...,...,...
95,kontrol,striktur uretra
96,kontrol post op tak ada keluhan,stricture uretra
97,kontrol sempat membaik &,dermatitis numular &
98,gatal paha makin banyak & 1th & & nbsptx &,dermatitis numular &


In [ ]:
df_content.columns = ['FS_ANAMNESA', 'FS_DIAGNOSA']

# Process

In [ ]:
# make a list of all the words in FS_ANAMNESA and FS_DIAGNOSA
word_list_anam = []
word_list_diag = []
for i in df_content.index:
  word_list_anam = word_list_anam + word_tokenize(df_content['FS_ANAMNESA'][i])
  word_list_diag = word_list_diag + word_tokenize(df_content['FS_DIAGNOSA'][i])

In [ ]:
word_list_anam

['demam',
 '2',
 'hari',
 'pusing',
 'makan',
 'minum',
 'baik',
 'lemas',
 'kontrol',
 'post',
 'opname',
 'kontrol',
 'post',
 'opname',
 'panas',
 '&',
 '3',
 'hari',
 'miuntah',
 'pilek',
 '&',
 'kontrol',
 'batuk',
 'panas',
 'pilek',
 '&',
 'batuk',
 'sering',
 'kambuh',
 'batuk',
 'pilek',
 'demam',
 'kemerahan',
 'badan',
 'batuk',
 'sesek',
 'nafas',
 '&',
 'rujukan',
 'klinik',
 'laras',
 'hati',
 'gigi',
 'kiri',
 'atas',
 'berlubang',
 'melanjutkan',
 'perawatan',
 'gigi',
 'sebelah',
 'kiri',
 'atas',
 'melanjutkan',
 'perawatan',
 'pusing',
 'lemas',
 'batuk',
 'demam',
 '1',
 'hari',
 'dahak',
 'warna',
 'kuning',
 'kental',
 'paracetamol',
 'tidak',
 'membaik',
 'minta',
 'surat',
 'keterangan',
 'sehat',
 '7',
 'bebas',
 'narkoba',
 'kontrol',
 'evaluasi',
 'bmp',
 'kontrol',
 '4',
 'kontrol',
 'et',
 'konsultasi',
 'hasil',
 'laborat',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol',
 'te',
 'kontrol',
 'kontrol',
 'kontrol',
 'kon

In [ ]:
# word_list_anam[word_list_anam == 'ma']

In [ ]:
word_anam = pd.Series(word_list_anam)
word_diag = pd.Series(word_list_diag)

In [ ]:
word_anam

0       demam
1           2
2        hari
3      pusing
4       makan
        ...  
724         &
725         &
726         2
727     tahun
728         &
Length: 729, dtype: object

In [ ]:
word_anam = word_anam.drop_duplicates()
word_diag = word_diag.drop_duplicates()

In [ ]:
# size of the corpus

global N
N=len(data_index) # 1399

# create tf-idf matrix for documents

## TF-IDF

### FS_ANAMNESA

In [ ]:
columns_anam = ['index_terms_anam']
for i in (df_content.index):
  columns_anam.append('doc_'+str(i))

# initiate the tf matrix
tf_matrix_anam = pd.DataFrame(columns=columns_anam)

# fill up the index terms column using the unique words
tf_matrix_anam['index_terms_anam']=word_anam

# initialize with all zeros
tf_matrix_anam.fillna(0, inplace=True)

# create terms as indices
tf_matrix_anam.set_index(['index_terms_anam'], inplace=True)

# fill tf_matrix with the frequency of each term for every docs
count = 0
for j in word_list_anam:
  count += 1
  for k in df_content.index:
    anam = df_content['FS_ANAMNESA'][k] 
    docid = k
    f = anam.count(j)
    if f !=0:
      tf_matrix_anam['doc_'+str(docid)][j] = f
  print("done for word:",j, "at level", count)
print("Weighted term frequency matrix completed.") 

done for word: demam at level 1
done for word: 2 at level 2
done for word: hari at level 3
done for word: pusing at level 4
done for word: makan at level 5
done for word: minum at level 6
done for word: baik at level 7
done for word: lemas at level 8
done for word: kontrol at level 9
done for word: post at level 10
done for word: opname at level 11
done for word: panas at level 12
done for word: & at level 13
done for word: 3 at level 14
done for word: miuntah at level 15
done for word: pilek at level 16
done for word: batuk at level 17
done for word: sering at level 18
done for word: kambuh at level 19
done for word: kemerahan at level 20
done for word: badan at level 21
done for word: sesek at level 22
done for word: nafas at level 23
done for word: rujukan at level 24
done for word: klinik at level 25
done for word: laras at level 26
done for word: hati at level 27
done for word: gigi at level 28
done for word: kiri at level 29
done for word: atas at level 30
done for word: berluban

In [ ]:
for i in range(0,N):
    tf_matrix_anam['doc_'+str(i)] = tf_matrix_anam['doc_'+str(i)] / max(tf_matrix_anam['doc_'+str(i)])
    
# append a column D at the end of tf
sLength = len(tf_matrix_anam.index)
D = pd.Series(np.zeros(sLength))
tf_matrix_anam = tf_matrix_anam.assign(D=D.values)
# append an IDF column
IDF = pd.Series(np.zeros(sLength))
tf_matrix_anam = tf_matrix_anam.assign(IDF=IDF.values)
# fill the nonzero count at D

In [ ]:
tf_matrix_anam

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_92,doc_93,doc_94,doc_95,doc_96,doc_97,doc_98,doc_99,D,IDF
index_terms_anam,,,,,,,,,,,,,,,,,,,,,
demam,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.166667,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.0,0.0
hari,0.166667,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.166667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
pusing,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
makan,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sempat,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.000000,0.0,0.0
paha,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.142857,0.000000,0.0,0.0
makin,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.142857,0.000000,0.0,0.0


In [ ]:
tf_matrix_anam['IDF'][tf_matrix_anam.index[21]]

0.0

In [ ]:
"""
    computing term frequecny and inverce-document frequency
    for every index terms. two columns are being added for this purpose.
"""
for indices in range(len(tf_matrix_anam.index)):
  di = sum(tf_matrix_anam.loc[tf_matrix_anam.index[indices]] > 0)
  print(indices, N, di)
  # update the di values |di|=non zero entries of a term w.r.t a doc
  tf_matrix_anam['D'][tf_matrix_anam.index[indices]] = di
  # IDF computation
  tf_matrix_anam['IDF'][tf_matrix_anam.index[indices]] = math.log(N/di,10)
  
    
"""
    tf-idf computation phase. compute tf-idf for each entry of the data 
"""
global A
A = tf_matrix_anam
for i in range(len(tf_matrix_anam.index)):
  A.loc[A.index[i]][0:N+1] = A.loc[A.index[i]][0:N+1] * A.IDF[i]
        
#query = "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft"

0 100 4
1 100 26
2 100 10
3 100 2
4 100 2
5 100 2
6 100 6
7 100 2
8 100 34
9 100 7
10 100 3
11 100 5
12 100 23
13 100 18
14 100 1
15 100 4
16 100 7
17 100 2
18 100 3
19 100 2
20 100 3
21 100 1
22 100 1
23 100 3
24 100 1
25 100 1
26 100 1
27 100 5
28 100 4
29 100 3
30 100 1
31 100 2
32 100 3
33 100 1
34 100 32
35 100 2
36 100 1
37 100 1
38 100 1
39 100 1
40 100 6
41 100 4
42 100 1
43 100 1
44 100 1
45 100 1
46 100 8
47 100 1
48 100 1
49 100 1
50 100 1
51 100 17
52 100 11
53 100 2
54 100 1
55 100 1
56 100 14
57 100 2
58 100 1
59 100 1
60 100 1
61 100 2
62 100 1
63 100 9
64 100 8
65 100 2
66 100 11
67 100 1
68 100 1
69 100 1
70 100 2
71 100 1
72 100 3
73 100 2
74 100 1
75 100 1
76 100 1
77 100 1
78 100 3
79 100 9
80 100 11
81 100 4
82 100 1
83 100 10
84 100 93
85 100 84
86 100 87
87 100 14
88 100 10
89 100 5
90 100 2
91 100 1
92 100 1
93 100 5
94 100 4
95 100 4
96 100 2
97 100 1
98 100 1
99 100 1
100 100 2
101 100 1
102 100 1
103 100 1
104 100 2
105 100 1
106 100 1
107 100 1
108 100 1
109

'\n    tf-idf computation phase. compute tf-idf for each entry of the data \n'

### FS_DIAGNOSA

In [ ]:
columns_diag = ['index_terms_diag']
for i in (df_content.index):
  columns_diag.append('doc_'+str(i))

# initiate the tf matrix
tf_matrix_diag = pd.DataFrame(columns=columns_diag)

# fill up the index terms column using the unique words
tf_matrix_diag['index_terms_diag']=word_diag

# initialize with all zeros
tf_matrix_diag.fillna(0, inplace=True)

# create terms as indices
tf_matrix_diag.set_index(['index_terms_diag'], inplace=True)

# fill tf_matrix with the frequency of each term for every docs
count = 0
for j in word_list_diag:
  count += 1
  for k in df_content.index:
    diag = df_content['FS_DIAGNOSA'][k] 
    docid = k
    f = diag.count(j)
    if f !=0:
      tf_matrix_diag['doc_'+str(docid)][j] = f
  print("done for word:",j, "at level", count)
print("Weighted term frequency matrix completed.") 

done for word: febris at level 1
done for word: hari at level 2
done for word: 2 at level 3
done for word: isk at level 4
done for word: post at level 5
done for word: rawat at level 6
done for word: inap at level 7
done for word: & at level 8
done for word: isk at level 9
done for word: post at level 10
done for word: rawat at level 11
done for word: inap at level 12
done for word: & at level 13
done for word: obs at level 14
done for word: febris at level 15
done for word: 3 at level 16
done for word: ec at level 17
done for word: & at level 18
done for word: kontrol at level 19
done for word: bkb at level 20
done for word: alergi at level 21
done for word: & at level 22
done for word: rfa at level 23
done for word: bkb at level 24
done for word: ec at level 25
done for word: alergi at level 26
done for word: susp at level 27
done for word: asma at level 28
done for word: scarlet at level 29
done for word: fever at level 30
done for word: & at level 31
done for word: asma at level 32

In [ ]:
tf_matrix_diag

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_90,doc_91,doc_92,doc_93,doc_94,doc_95,doc_96,doc_97,doc_98,doc_99
index_terms_diag,,,,,,,,,,,,,,,,,,,,,
febris,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hari,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
isk,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
post,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uretra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
stricture,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
asteatotik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
len(tf_matrix_diag)

174

In [ ]:
N

100

In [ ]:
for i in range(0,N):
    tf_matrix_diag['doc_'+str(i)] = tf_matrix_diag['doc_'+str(i)] / max(tf_matrix_diag['doc_'+str(i)])
    
# append a column D at the end of tf
sLength = len(tf_matrix_diag.index)
D = pd.Series(np.zeros(sLength))
tf_matrix_diag = tf_matrix_diag.assign(D=D.values)
# append an IDF column
IDF = pd.Series(np.zeros(sLength))
tf_matrix_diag = tf_matrix_diag.assign(IDF=IDF.values)
# fill the nonzero count at D

In [ ]:
tf_matrix_diag

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_92,doc_93,doc_94,doc_95,doc_96,doc_97,doc_98,doc_99,D,IDF
index_terms_diag,,,,,,,,,,,,,,,,,,,,,
febris,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0
hari,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0
2,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0
isk,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0
post,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uretra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.25,0.25,0.25,0.0,0.0,0.000000,0.0,0.0
stricture,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.25,0.0,0.0,0.000000,0.0,0.0
asteatotik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.166667,0.0,0.0


In [ ]:
"""
    computing term frequecny and inverce-document frequency
    for every index terms. two columns are being added for this purpose.
"""
for indices in range(len(tf_matrix_diag.index)):
  di = sum(tf_matrix_diag.loc[tf_matrix_diag.index[indices]] > 0) 
  # update the di values |di|=non zero entries of a term w.r.t a doc
  tf_matrix_diag['D'][tf_matrix_diag.index[indices]] = di
  # IDF computation
  tf_matrix_diag['IDF'][tf_matrix_diag.index[indices]] = math.log(N/di,10)
    
"""
    tf-idf computation phase. compute tf-idf for each entry of the data 
"""
global B
B = tf_matrix_diag
for i in range(len(tf_matrix_diag.index)):
  B.loc[B.index[i]][0:N+1] = B.loc[B.index[i]][0:N+1] * B.IDF[i]

In [ ]:
B

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_92,doc_93,doc_94,doc_95,doc_96,doc_97,doc_98,doc_99,D,IDF
index_terms_diag,,,,,,,,,,,,,,,,,,,,,
febris,0.849485,0.000000,0.000000,0.849485,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.000000,3.397940,1.698970
hari,0.849485,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.000000,3.397940,1.698970
2,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.000000,10.000000,1.000000
isk,0.000000,0.849485,0.849485,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.000000,3.397940,1.698970
post,0.000000,0.698970,0.698970,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.000000,5.591760,1.397940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uretra,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.38072,0.38072,0.38072,0.0,0.0,0.000000,4.568636,1.522879
stricture,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.50000,0.0,0.0,0.000000,2.000000,2.000000
asteatotik,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.333333,2.000000,2.000000


# User Query

In [ ]:
global K
K = 10

In [ ]:
def user_query_anam_processing(query): 
  q_mat = pd.DataFrame(columns={'index_terms','q'})
  q_mat.index_terms = word_anam
  q_mat.fillna(0,inplace=True)
  q_mat.set_index('index_terms', inplace=True)
          
  # intersect query and index terms
  valid_q = list(set(query).intersection(list(A.index)))
          
  for m in range(len(valid_q)):
    for n in range(len(A)):
      if valid_q[m] == A.index[n]:
        q_mat.q[n] = 1
              
  # print(q_mat.q, A.IDF)            
  q_mat.q = q_mat.q * A.IDF
  # #print(q_mat.q)
  scores = np.zeros(N)
  # # the first position is not needed, so keeping an invalid value there
  scores[0] = -5555555
          
  # """
  #  cosine similarity function:
  #  if any of the denominator or numerator is zero the score is set to 0
  # """
  for i in range(0,N):
    vec_dot = np.dot(q_mat.q, A['doc_'+str(i)])
    vec_norm = (LA.norm(q_mat.q) * LA.norm(A['doc_'+str(i)]))
    if vec_dot == 0 or vec_norm == 0:
      scores[i] = 0.0
    if vec_dot > 0 or vec_norm > 0:
      scores[i] = vec_dot / vec_norm

  global top_scores, top_docs 
  
  top_scores = sorted(scores,reverse=True)
  top_n_scores = top_scores[:K]
  top_docs = np.array(np.argsort(scores))
  top_docs = top_docs[::-1] # reverse
  top_n_docs = top_docs[:K]

  global ret_doc_id
  ret_doc_id = pd.DataFrame(columns=['Ranking', 'Scores', 'Document Number', 'FS_ANAMNESA', 'FS_TINDAKAN', 'FS_TERAPI', 'FS_CATATAN_FISIK',
        'FS_DIAGNOSA'])
  
  print('Rank'+'\t'+'Scores'+'\t' +'Document number '+ '\t'+ 'ANAMNESA '+'\t' + 'TINDAKAN '+'\t' + 'TERAPI '+'\t' + 'CATATAN FISIK '+'\t' + 'DIAGNOSA'+'\n' )
  for p in range(K):
    ret_doc_id = ret_doc_id.append({'Ranking': p+1, 'Scores': round(top_n_scores[p],3), 'Document Number': top_n_docs[p], 'FS_ANAMNESA': data.loc[top_n_docs[p]].FS_ANAMNESA, 'FS_TINDAKAN': data.loc[top_n_docs[p]].FS_TINDAKAN, 'FS_TERAPI': data.loc[top_n_docs[p]].FS_TERAPI, 'FS_CATATAN_FISIK': data.loc[top_n_docs[p]].FS_CATATAN_FISIK, 'FS_DIAGNOSA': data.loc[top_n_docs[p]].FS_DIAGNOSA}, ignore_index=True)
    ret_doc_id.set_index('Ranking', inplace = True)

  ret_doc_id.set_index('Ranking', inplace = True)
  display(ret_doc_id)

In [ ]:
def user_query_diag_processing(query): 
  q_mat = pd.DataFrame(columns={'index_terms','q'})
  q_mat.index_terms = word_diag
  q_mat.fillna(0,inplace=True)
  q_mat.set_index('index_terms', inplace=True)
          
  # intersect query and index terms
  valid_q = list(set(query).intersection(list(B.index)))
          
  for m in range(len(valid_q)):
    for n in range(len(B)):
      if valid_q[m] == B.index[n]:
        q_mat.q[n] = 1
              
  # print(q_mat.q, A.IDF)            
  q_mat.q = q_mat.q * B.IDF
  # #print(q_mat.q)
  scores = np.zeros(N)
  # # the first position is not needed, so keeping an invalid value there
  scores[0] = -5555555
          
  # """
  #  cosine similarity function:
  #  if any of the denominator or numerator is zero the score is set to 0
  # """

  for i in range(0,N):
    vec_dot = np.dot(q_mat.q, B['doc_'+str(i)])
    vec_norm = (LA.norm(q_mat.q) * LA.norm(B['doc_'+str(i)]))
    if vec_dot == 0 or vec_norm == 0:
      scores[i] = 0.0
    if vec_dot > 0 or vec_norm > 0:
      scores[i] = vec_dot / vec_norm 

  global top_scores, top_docs 

  top_scores = sorted(scores,reverse=True)
  top_n_scores = top_scores[:K]
  top_docs = np.array(np.argsort(scores))
  top_docs = top_docs[::-1] # reverse
  top_n_docs = top_docs[:K]

  print(top_scores)
  print(top_docs)

  global ret_doc_id
  ret_doc_id = pd.DataFrame(columns=['Ranking', 'Scores', 'Document Number', 'FS_ANAMNESA', 'FS_TINDAKAN', 'FS_TERAPI', 'FS_CATATAN_FISIK',
        'FS_DIAGNOSA'])
  
  print('Rank'+'\t'+'Scores'+'\t' +'Document number '+ '\t'+ 'ANAMNESA '+'\t' + 'TINDAKAN '+'\t' + 'TERAPI '+'\t' + 'CATATAN FISIK '+'\t' + 'DIAGNOSA'+'\n' )
  for p in range(K):
    ret_doc_id = ret_doc_id.append({'Ranking': p+1, 'Scores': round(top_n_scores[p],3), 'Document Number': top_n_docs[p], 'FS_ANAMNESA': data.loc[top_n_docs[p]].FS_ANAMNESA, 'FS_TINDAKAN': data.loc[top_n_docs[p]].FS_TINDAKAN, 'FS_TERAPI': data.loc[top_n_docs[p]].FS_TERAPI, 'FS_CATATAN_FISIK': data.loc[top_n_docs[p]].FS_CATATAN_FISIK, 'FS_DIAGNOSA': data.loc[top_n_docs[p]].FS_DIAGNOSA}, ignore_index=True)

  ret_doc_id.set_index('Ranking', inplace = True)
  display(ret_doc_id)

In [ ]:
print('Apa yang akan Anda pilih?')
print('Anamnesa ketikkan 0')
print('Diagnosa ketikkan 1')

inputan = input("yang Anda pilih adalah ")

while True:
  user_query = word_tokenize(input("Search query >> "))
  if user_query == []:
    sys.exit()
  elif inputan == 0:
    user_query_anam_processing(user_query)
  else:
    user_query_diag_processing(user_query)

Apa yang akan Anda pilih?
Anamnesa ketikkan 0
Diagnosa ketikkan 1
yang Anda pilih adalah 1
Search query >> neurosa
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[99 36 26 27 28 29 30 31 32 33 34 35 37 98 38 39 40 41 42 43 44 45 46 47
 25 24 23 22  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 48 49 50 75 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 76 74 51 73 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72  0]
Rank	Scores	Document number 	ANAMNESA 	TINDAKAN 	TERAPI 	CATATAN FISIK 	DIAGNO

,Scores,Document Number,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
Ranking,,,,,,,
1,0.0,99,"<p>-kulit merah, gatal di kaki dan tangan&nbsp...",NaN,<p>R/ Metylpredenisolon 8 mg tab no XV</p> <p...,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>Dermatitis Asteatotik</p> <p>PSoriasis Vul...
2,0.0,36,"<p>konsultasi program hamil,sdh nikah 3th.</p>...",NaN,<p>protira no 10</p> <p>s1dd1 mulai 17 april ...,"<p>Suhu : C,Nadi : 100 x/menit,Respirasi : x/m...","<p>konsultasi IA, turunkan BB.&nbsp;</p> <p>S..."
3,0.0,26,<p>KONTROL</p>,NaN,<p>Levofloxacin 500mg tab V</p> <p>s 1x1</p> ...,"<p>Suhu : C,Nadi : 87 x/menit,Respirasi : x/me...",<p>TE</p>
4,0.0,27,<p>KONTROL TE</p>,NaN,<p>Cytodrox 500mg tab no XXX</p> <p>s 1x1&nbs...,"<p>Suhu : C,Nadi : 94 x/menit,Respirasi : x/me...",<p>Tromvbositosis esensial&nbsp;</p>
5,0.0,28,<p>KONTROL</p> <p>pembesaran kelenjar getah b...,NaN,<p>Levofloxacin 1x500mg</p>,"<p>Suhu : C,Nadi : 87 x/menit,Respirasi : x/me...",<p>TE</p>
6,0.0,29,<p>g4p3ab1 (hpm 11/06/18) kell:kadang nyeri pe...,<p>GS intrauterine GS 17 mm sesuai hamil 5 mg</p>,<p>anemolat 1mg no 20</p> <p>S1dd1</p> <p>Ca...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>6-7 mg</p>
7,0.0,30,<p>kadang kenceng kenceng.hpm 11 juni 18.uk23+...,"<p>janin tunggal, presbo, laki laki, djj+, ger...",<p>folamil genio no 30</p> <p>S1dd1</p>,"<p>Suhu : 36 C,Nadi : x/menit,Respirasi : 20 x...",<p>hamil 23 mg</p>
8,0.0,31,<p>g4p2a1 hpht 11-6-2018 oeriksa hamil t a k</p>,"<p>janin tunggal, preskep, gerak +, ketuban ba...",<p>Folamil No 10</p> <p>s1dd1</p> <p>Ossovit...,"<p>Suhu : C,Nadi : 89 x/menit,Respirasi : x/me...",<p>hamil 37 mg</p>
9,0.0,32,"<p>G4P2A1, HPHT= 11-6-19, periksa kehamilan</p>","<p>janin tunggal, djj+, gerak +, ketuban baik,...",NaN,"<p>Suhu : C,Nadi : 90 x/menit,Respirasi : x/me...","<p>hamil 36 mg, preskep</p>"


Search query >> febris
[0.5501977377418493, 0.47482132292821266, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 0  3 37 27 28 29 30 31 32 33 34 35 36 38 25 39 40 41 42 43 44 45 46 47
 26 24 98 23  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 22 48 99 50 75 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 76 74 51 73 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72 49]
Rank	Scores	Document number 	ANAMNESA 	TINDAKAN 	TERAPI 	CATATAN FISIK 	DIAGNOSA



,Scores,Document Number,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
Ranking,,,,,,,
1,0.550,0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",NaN,<p>paracetamol</p> <p>ranitidin</p> <p>imbos...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>febris hari 2</p>
2,0.475,3,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Obs febris hr ke 3 ec&nbsp;</p>
3,0.000,37,<p>Ingin membersihkan karang gigi</p> <p>P0A0...,<p>scaling ra rb</p> <p>USG uterus dbn dengan...,<p>primolt no 36</p> <p>S3dd1 mulai 21 februa...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>calculus</p> <p>penundaan haid</p>
4,0.000,27,<p>KONTROL TE</p>,NaN,<p>Cytodrox 500mg tab no XXX</p> <p>s 1x1&nbs...,"<p>Suhu : C,Nadi : 94 x/menit,Respirasi : x/me...",<p>Tromvbositosis esensial&nbsp;</p>
5,0.000,28,<p>KONTROL</p> <p>pembesaran kelenjar getah b...,NaN,<p>Levofloxacin 1x500mg</p>,"<p>Suhu : C,Nadi : 87 x/menit,Respirasi : x/me...",<p>TE</p>
6,0.000,29,<p>g4p3ab1 (hpm 11/06/18) kell:kadang nyeri pe...,<p>GS intrauterine GS 17 mm sesuai hamil 5 mg</p>,<p>anemolat 1mg no 20</p> <p>S1dd1</p> <p>Ca...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>6-7 mg</p>
7,0.000,30,<p>kadang kenceng kenceng.hpm 11 juni 18.uk23+...,"<p>janin tunggal, presbo, laki laki, djj+, ger...",<p>folamil genio no 30</p> <p>S1dd1</p>,"<p>Suhu : 36 C,Nadi : x/menit,Respirasi : 20 x...",<p>hamil 23 mg</p>
8,0.000,31,<p>g4p2a1 hpht 11-6-2018 oeriksa hamil t a k</p>,"<p>janin tunggal, preskep, gerak +, ketuban ba...",<p>Folamil No 10</p> <p>s1dd1</p> <p>Ossovit...,"<p>Suhu : C,Nadi : 89 x/menit,Respirasi : x/me...",<p>hamil 37 mg</p>
9,0.000,32,"<p>G4P2A1, HPHT= 11-6-19, periksa kehamilan</p>","<p>janin tunggal, djj+, gerak +, ketuban baik,...",NaN,"<p>Suhu : C,Nadi : 90 x/menit,Respirasi : x/me...","<p>hamil 36 mg, preskep</p>"


Search query >> dermatitis
[0.5154310453248954, 0.5154310453248954, 0.45659980007025136, 0.44536000409361887, 0.44536000409361887, 0.32010525043203336, 0.24982744155569328, 0.2425631035108675, 0.2425631035108675, 0.2425631035108675, 0.1929102283489879, 0.17129162363579653, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[97 98 40 44 39 68 62 47 46 48 99 41  9 38 31 32 33  6 34 35 36 37  5 10
  4 42 43  3 45  2  1 30 29  7 28 11 12 13 14 15 16 17 18 19 20 21 22 23
  8 25 26 27 24 49 50 86 77 78 79 80 81 82 83 84 85 87 51 88 89 90 91 92
 93 94 95 96 76 75 74 73 52 53 54 55 56 57 58 59 60 61 63 64 65 66 

,Scores,Document Number,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
Ranking,,,,,,,
1,0.515,97,<p>KONTROL sempat membaik&nbsp;</p>,NaN,<p>R/ Hexilon 8 mg tab no XV</p> <p>S 1-0-0&n...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
2,0.515,98,<p>GATAL DI PAHA makin banyak&nbsp; 1th&nbsp;<...,NaN,<p>R/ Hexilon 8 mg tab no X</p> <p>S 1-0-0&nb...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
3,0.457,40,"<p>-gatal di tangan dan kaki,&nbsp;</p>",NaN,<p>R/ Toras 4 mg tab no X</p> <p>S 1-0-0&nbsp...,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>Dermatitis Atopik</p>
4,0.445,44,<p>gatal di wajah dan leher kambuh&nbsp;</p>,NaN,<p>R/ Toras 4 mg tab no XV&nbsp;</p> <p>S 2 d...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Atopik&nbsp;</p>
5,0.445,39,<p>gatal di tangan dan kaki masih kasar&nbsp;</p>,NaN,<p>R/ Toras 4 mg tab no XX&nbsp;</p> <p>S 1-1...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Atopik&nbsp;</p>
6,0.320,68,<p>-bintik2 merah dan gatal di tangan dan kaki...,NaN,<p>R/ Lortadine tab no X</p> <p>S 0-0-1</p> ...,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>DermatitisAtopik&nbsp;</p>
7,0.250,62,<p>gatal gatal di tangan dan kaki kulit aberna...,NaN,<p>R/ Cefixim 100 mg tab no X</p> <p>' S2 dd ...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Atoik&nbsp;</p> <p>Impetigo kru...
8,0.243,47,<p>tangan gatal</p>,NaN,<p>R/ Ketokonazol gr 20&nbsp;</p> <p>&nbsp;Mo...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Tinea Korporis&nbsp;</p> <p>&nbsp;DD Derma...
9,0.243,46,<p>tangan gatal</p>,NaN,<p>R/ Ketokonazol gr 20&nbsp;</p> <p>&nbsp;Mo...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Tinea Korporis&nbsp;</p> <p>&nbsp;DD Derma...


Search query >> 


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
ret_doc_id['Scores'].loc[ret_doc_id.index == 1]

# df.loc[df['col1'] == value]

Ranking
1    0.515
Name: Scores, dtype: float64

# Evaluation

## Precision

In [ ]:
def prec(ret):
  rel_items_retrieved = 14
  ret_items = 10
  
  for a in range(1, len(ret)+1):
    ret_items += 1
    if(ret['Scores'].loc[ret.index == a] > 0).all() :
      rel_items_retrieved += 1
    elif(ret['Scores'].loc[ret.index == a] == 0).all() :
      continue

  precision = rel_items_retrieved / ret_items
  print(precision)

In [ ]:
prec(ret_doc_id)

1.0


## Recall

In [ ]:
def rec(ret, document):
  rel_items_retrieved = 0
  ret_items = 0
  rel_items = 0

  for a in range(1, len(ret)+1):
    ret_items += 1
    if(ret['Scores'].loc[ret.index == a] > 0).all() :
      rel_items_retrieved += 1
    elif(ret['Scores'].loc[ret.index == a] == 0).all() :
      continue

  for b in range(len(document)):
    if(document[b] != 0):
      rel_items += 1
    elif(document[b] == 0):
      break

  recall = rel_items_retrieved / rel_items
  print(recall, rel_items_retrieved, rel_items)

In [ ]:
len(top_scores)

100

In [ ]:
rec(ret_doc_id, top_scores)

0.8333333333333334 10 12
